In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pulp import *

## Reading in Dataset

### Fixed Cost of each WTP

In [23]:
fixed_cost = pd.read_excel("../dataset/fixed_cost.xlsx", index_col=0)
fixed_cost.head()

,Low,High
WTP,,
USA,6500,9500
Germany,4980,7270
Japan,6230,9100
Brazil,3230,4730
India,2110,6160


### Capacity of each WTP

In [74]:
capacity = pd.read_excel("../dataset/capacity.xlsx", index_col=0)
capacity.head()

,Capacity
WTP,
USA,1000
Germany,1000
Japan,1000
Brazil,1000
India,1000


### Variable Cost for each combination of (DMZ, WTP)

In [25]:
variable_cost = pd.read_excel("../dataset/variable_costs.xlsx", index_col=0)
variable_cost.head()

,USA,Germany,Japan,Brazil,India
Variable Costs ($/Unit),,,,,
DMZ1,12,12,12,12,12
DMZ2,13,13,13,13,13
DMZ3,10,10,10,10,10
DMZ4,8,8,8,8,8
DMZ5,5,5,5,5,5


### Transport Cost for each combination of (DMZ, WTP)

In [26]:
transport_cost = pd.read_excel("../dataset/freight_costs.xlsx", index_col=0)
transport_cost.head()

,USA,Germany,Japan,Brazil,India
Freight Costs ($/Container),,,,,
DMZ1,0,12250,1100,16100,8778
DMZ2,13335,0,8617,20244,10073
DMZ3,15400,22750,0,43610,14350
DMZ4,16450,22050,28000,0,29750
DMZ5,13650,15400,24500,29400,0


### Demand for each DMZ

In [48]:
demand = pd.read_excel("../dataset/demand.xlsx", index_col=0)
demand.head()

,Demand
DMZ,
DMZ1,2800000
DMZ2,90000
DMZ3,1700000
DMZ4,145000
DMZ5,160000


## Optimisation

In [138]:
# List of all the WTPs
wtp = list(capacity.index)

# List of all the DMZs
dmz = list(demand.index)

# List of (DMZ, WTP) pairs
dmz_wtp_pairs = [(d, w) for d in dmz for w in wtp]

In [139]:
demand.loc[dmz[0], "Demand"]

2800000

In [140]:
# Creating the Linear Optimisation Class
model = LpProblem("Optimising water supply", LpMinimize)

In [141]:
# Creating Decision Variables
output = LpVariable.dicts("Volume", dmz_wtp_pairs, lowBound=0, upBound=None, cat='continuous')

In [142]:
# Define the Objective Function
model += \
     lpSum([fixed_cost.loc[w, "High"] * 1000 for w in wtp]) + \
     lpSum([(variable_cost.loc[d, w] + transport_cost.loc[d, w]) * output[(d, w)] for d in dmz for w in wtp])

In [143]:
# Adding Constraints

## Meet demand for each DMZ
for d in dmz:
    model += lpSum([output[(d, w)] for w in wtp]) == demand.loc[d, "Demand"]

## Within the WTP capacity
for w in wtp:
    model += lpSum([output[(d, w)] for d in dmz]) <= capacity.loc[w, "Capacity"] * 1000

## No linkage constraint



In [144]:
# Solve the model
model.solve()

1

In [145]:
print("Total Costs = {:,} ($/Month)".format(int(value(model.objective))))
print('\n' + "Status: {}".format(LpStatus[model.status]))

Total Costs = 34,586,910,000 ($/Month)

Status: Optimal


## Results

In [111]:
dict_wtp = {}
dict_dmz = {}

In [146]:
df = pd.DataFrame(0, index=dmz, columns = wtp)
df

,USA,Germany,Japan,Brazil,India
DMZ1,0,0,0,0,0
DMZ2,0,0,0,0,0
DMZ3,0,0,0,0,0
DMZ4,0,0,0,0,0
DMZ5,0,0,0,0,0


In [147]:
# Getting the results
for v in model.variables():
    # print(v.name, v.varValue)
    name = v.name.replace("Volume_", "").replace("_", "")
    # print(name)

    combi = eval(name)
    # print(combi[0])

    dmz = combi[0]
    wtp = combi[1]
    volume = v.varValue

    # print("DMZ: ", dmz, " ", "WTP: ", wtp, " ", "Value: ", volume)

    df.loc[dmz, wtp] = volume


In [148]:
df

,USA,Germany,Japan,Brazil,India
DMZ1,1000000,910000,0,750000,140000
DMZ2,0,90000,0,0,0
DMZ3,0,0,1000000,0,700000
DMZ4,0,0,0,145000,0
DMZ5,0,0,0,0,160000
